# BPSD: Export Beat Positions and Time Signatures

Johannes Zeitler (johannes.zeitler@audiolabs-erlangen.de), 2024

In [1]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

from itertools import groupby
import scipy

In [6]:
ann_audio_note_dir = os.path.join("../", "2_Annotations", "ann_audio_note")
beat_out_dir = os.path.join("../", "2_Annotations", "ann_audio_beat")
timesig_out_dir = os.path.join("../", "2_Annotations", "ann_score_timesignature")

In [7]:
pieces = [f for f in os.listdir(ann_audio_note_dir) if ".csv" in f]

pieces.sort()

In [8]:
for piece in tqdm(pieces):
    noteIn = pd.read_csv(os.path.join(ann_audio_note_dir, piece), sep=";")
    noteIn.sort_values(by="start_meas", inplace=True)
    nMeasures = int(max(noteIn.end_meas))
    
    measures = np.arange(nMeasures)
    meas_timesig = []
    
    for meas in measures:
        notes_meas = noteIn[noteIn["start_meas"].astype(int) == meas]
        
        if len(notes_meas) > 0:
            last_timeSig = list(notes_meas["timesig"])[0]
            meas_timesig.append((meas, last_timeSig))
        elif meas == 0: continue
        else:
            meas_timesig.append((meas, last_timeSig))
        
    beats = []
    meas_list = []
    timesig_list = []
    for meas, timesig in meas_timesig:
        meas_list.append(meas)
        timesig_list.append(timesig)
        nBeats = int(timesig.split("/")[0])
        for beatIdx in range(nBeats):
            beats.append(meas + 1/nBeats*beatIdx)
            
            
    beats = [b for b in beats if b >= (min(noteIn.start_meas)-1e-3)] 
    
    start_pairs = [[start_m, start_s] for start_m, start_s in zip(noteIn.start_meas, noteIn.start)]
    start_pairs_unique = np.array([pair for pair,_ in groupby(start_pairs)])
    start_pairs_unique.sort(axis=0)        

    meas_to_time_annot = scipy.interpolate.interp1d(start_pairs_unique[:,0], start_pairs_unique[:,1], 
                                              kind='linear', bounds_error=False, fill_value='extrapolate')

    beat_times = meas_to_time_annot(beats)

    df_beats = pd.DataFrame({ "time":beat_times, "beat":beats})
    df_beats.to_csv(os.path.join(beat_out_dir, piece), sep=";", index=False, float_format="%07.03f")
    
    
    
    # save timesig only once
    performer = piece.split(".")[0].split("_")[-1]
    if performer == "WK64":
        sonata = "_".join(piece.split(".")[0].split("_")[:2])
        pd.DataFrame({"measure":meas_list,
                  "timesig":timesig_list}).to_csv(os.path.join(timesig_out_dir, sonata+".csv"),
                                                  sep=";", index=False, float_format="%07.03f")

  0%|          | 0/352 [00:00<?, ?it/s]